In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

loader = UnstructuredFileLoader("./document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}, and say in Korean Only.",
        ),
        ("human", "{question}"),
    ]
)

def chain_with_memory(question):
    chat_history = memory.load_memory_variables({}).get("chat_history", "")

    relevant_docs = retriever.get_relevant_documents(question)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    inputs = {
        "context": context,
        "question": question,
        "chat_history": chat_history,
    }

    prompt = prompt_template.format_messages(**inputs)
    response = llm(prompt)

    memory.save_context({"question": question}, {"answer": response.content})

    return response.content

# 결과 생성 및 출력
result1 = chain_with_memory("Aaronson 은 유죄인가요?")
result2 = chain_with_memory("그가 테이블에 어떤 메시지를 썼나요?")
result3 = chain_with_memory("Julia는 누구인가요?")

print(result1)
print(result2)
print(result3)


모르겠습니다.
그는 테이블에 "자유는 노예다"라고 썼고, 그 아래에 "둘과 둘은 다섯이다"라고 썼습니다. 그리고 나서 "신은 권력이다"라고 썼습니다.
Julia는 Winston의 사랑하는 사람입니다.
